# Classifying MRI Images of Prostate Cancer with Mxnet Gluon

Margaret Linan<br>
Content Editing in Progress

## Introduction to Prostate Cancer, MRI and Deep Learning

According to the American Academy of Family Physicians, magnetic resonance imaging (MRI) is a healthcare technology that is used to survey patient organs and other anatomical structures with the goal of finding signs of disease, such as tumors and abnormalities. Currently, biomedical research aims to increase the resolution of MRI techology so that medical professionals can view tissue at the micrometer scale.

The National Cancer Institute considers prostate cancer to be the most common cancer in men living in the United States. According to the American Cancer Society, there will be an estimated 161,360 new cases of prostate cancer in the United States in 2017, with 26,730 deaths.

![title](Prostate_MRI.jpg)
Source: https://www.alta-klinik.com/prostate-cancer.html

Earlier diagnostic approaches such as Prostate Specific Antigen test and Digital Rectal examination, did not have good accuracy and may have caused an increase in false positive diagnoses. Although technological advances such as Magnetic Resonance Imagining (MRI) have resulted in increased accuracy for prostate cancer diagnoses, it's overall specificity needs improvement. Thus, insilico approaches such as machine learning and deep learning have been the focus of much work over the last decade. The challenges that exist with machine learning is the need to accurately identify multiple features in MRI images and the radiologists varying level of knowledge and skill in accurately identifying these. Deep learning algorithms such as Convolutional Neural Networks (CNN), give radiologists the opportunity to leverage digital pathology as a tool to accurately diagnose prostate cancer and reduce the number of unnecessary biopsies.

### Deep Learning

The advent of GPU technology makes it possible to apply deep learning neural network models to large MRI image collections. 

#### MRIs and Convolutional Neural Networks (MXNET - GLUON)

Convolutional nets (CNN) are typically used to process data with a grid-like topology (1). Examples of this type of data include images, time-series, gene-expression (1,2). Examples of CNN applications include but are not limited to DNA sequence classification and modeling the sequence specificity of DNA-protein binding and biological relation extraction (3-5). 
The image below represents a CNN, in the feature learning stage convolutions are happening in parallel and linear activations are passed through the rectified linear activation (RELU) function (1). The pooling stages use a pooling function that then modifies the output of the layer (1). In the classification stage, new input is evaluated by the trained CNN and a classification prediction is made.

![title](plot_nature2.jpg)
Source: Nature Scientific Reports. doi:10.1038/s41598-017-15720-y

## The Cancer Imaging Archive 

The Cancer Imaging Archive (TCIA), is described on its website as the following: TCIA is a service which de-identifies and hosts a large archive of medical images of cancer accessible for public download. The data are organized as “Collections”, typically patients related by a common disease (e.g. lung cancer), image modality (MRI, CT, etc) or research focus. DICOM is the primary file format used by TCIA for image storage. Supporting data related to the images such as patient outcomes, treatment details, genomics, pathology, and expert analyses are also provided when available.

![title](image1.png)

## Let's begin the Deep Learning Lab

In [1]:
## MXNET Convolutional Neural Networks Code
## Source:  http://gluon.mxnet.io/chapter04_convolutional-neural-networks/cnn-gluon.html
## Note: Must make the mxnet library
import os, sys
import numpy as np
from PIL import Image
import glob
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd, gluon
mx.random.seed(1)

## Set the context 
ctx = mx.gpu()

In [7]:
### Alternative code for importing labels and images + predictions
# source: https://github.com/BartyzalRadek/Multi-label-Inception-net/blob/master/label_image.py
# source: https://towardsdatascience.com/multi-label-image-classification-with-inception-net-cbb2ee538e30

SyntaxError: invalid syntax (<ipython-input-7-f1cf79cfbbe4>, line 3)

In [8]:
import tensorflow as tf
import sys

# change this as you see fit
image_path = sys.argv[1]

# Read in the image_data
image_data = tf.gfile.FastGFile(image_path, 'rb').read()

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line 
                   in tf.gfile.GFile("labels.txt")]

# Unpersists graph from file
with tf.gfile.FastGFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

with tf.Session() as sess:
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    
    predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})
    
    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]
    
    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        print('%s (score = %.5f)' % (human_string, score))
    

    filename = "results.txt"    
    with open(filename, 'a+') as f:
        f.write('\n**%s**\n' % (image_path))
        for node_id in top_k:
            human_string = label_lines[node_id]
            score = predictions[0][node_id]
f.write('%s (score = %.5f)\n' % (human_string, score))

NotFoundError: -f; No such file or directory

Specify the input and output
The input for this CNN model will be the MRI images of the lesions from the prostate cancer patients.
The output will be the labels, where "1" represents clinical significance and "0" represents no clinical significance.


In [1]:
batch_size = 64  # number of samples per batch
num_inputs = 231
num_outputs = 99

train_image_list = []
for filename in glob.glob('yourpath/*.bmp'): 
    train_im=Image.open(filename)
    train_image_list.append(train_im)

test_image_list = []
for filename in glob.glob('yourpath/*.bmp'): 
    test_im=Image.open(filename)
    test_image_list.append(test_im)
 
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)


train_data = gluon.data.DataLoader(train_im(train=True, transform=transform), batch_size, shuffle=True)
test_data = gluon.data.DataLoader(test_im(train=False, transform=transform), batch_size, shuffle=False)

SyntaxError: invalid syntax (<ipython-input-1-52bd3fa04f1f>, line 15)

### Convolutional Neural Network or Deep CNN

Now let's define a convolutional neural network. According to the documentation in MXNET Gluon:<br> 

<b>num_fc</b> = specifies the number of neurons in the dense layer.<br>
<b>num_outputs</b> = specifies the number of neurons in the<br> 
<b>Conv2D</b> = This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. If use_bias is True, a bias vector is created and added to the outputs. Finally, if activation is not None, it is applied to the outputs as well.<br>
<b>MaxPool2D</b> = Max pooling operation for two dimensional (spatial) data.<br>


In [ ]:
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
## The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

### Parameter initialization
In this situation, gluon is not actually initializing any parameters! Instead, it’s making a note of which 
initializer to associate with each parameter, even though its shape is not yet known. The parameters are 
instantiated and the initializer is called once we provide the network with some input.
source: http://gluon.mxnet.io/chapter03_deep-neural-networks/plumbing.html


In [ ]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

### Softmax Cross-Entropy Loss
The Softmax classifier is one of the most popular classifiers in the deep learning field. 
Cross-entropy loss represents the distance or difference between what the model believes 
the output distribution should be, and what the original distribution really is. 
Note: equations, explanations and images: https://deepnotes.io/softmax-crossentropy


In [2]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

NameError: name 'gluon' is not defined

### Optimizer
The purpose of the SGD optimizer is to find the minima or maxima by iteration, based on the learning rate.
SGD is one of the most used optimization algorithms in the deep learning field. According to Goodfellow, Bengio, Courville 2016, the learning rate is critical and needed because SGD gradient estimator generates noise (the random sample of m training examples) that remain even when the minimum has been reached. Although the learning rate can be chosen by using the trial and error method, Goodfellow, Bengio, Courville 2016 offers the following advice on how to choose the best learning rate:
 
  1. Monitor the the learning curves that plot the objective functions as a function of time
  2. If the learning rate is too large, the oscillations will be violent, with the cost function often increasing significantly
  3. Mild oscillations are more acceptable especially if you are training with a stochastic cost function such as the cost function 
     arising from the use of dropout (Monte Carlo dropout is suggested, see https://arxiv.org/pdf/1506.02158v6.pdf). 
  4. If the learning rate is too small, learning may be stuck with a high cost value
 
Final thoughts offered by Goodfellow, Bengio, Courville 2016:
  1. Typically, the optimal intial learning rate, in terms of total training time and the final cost value, is higher than the 
     learning rate that yields the best performance after the first 100 iterations or so
  2. Do not use a learning rate that is so high that it causes severe instability



In [ ]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

In [ ]:
## Write an evaluation loop to calculate the model's accuracy

def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [3]:
## Training Loop
## Epochs = 1,  means that there will be one full pass of the training data set. 
## Smoothing_constant = allows some control of the bias of the variance estimation,
##                      and thus effectively smoothes the estimate.

epochs = 1
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + smoothing_constant * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    
## We can now evaulate the accuracy of the model   
    
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))



ModuleNotFoundError: No module named 'mxnet'

### Precision-Recall Analysis

The Precision-Recall metric is used to evaluate the classifiers output quality.
The results from a precision-recall analysis indicate the classifiers success at accurately predicting the classes of 
the dataset.
 
Precision measures the relevancy of the result, while recall measures the amount of truly relevant results. 
Visualizing the results of the precision-recall analysis helps communicate the tradeoff between precisian and recall for different thresholds. 
A large area under the curve (AUC) value inducates that there is a high recall and precision. 

 High Precision ~ low false positive rate
 High Recall ~ low false negative rate 

When the precision analysis is able to prove that there is high precision and high recall for the classifier, it means 
that the classifier is accurate and is mostly returning all positive results. 

<b>General Expectations </b>

 - Classifiers with high recall but low precision returns many results, but most of its predicted labels are incorrect when  
   compared to the training labels. 
 - Classifiers with high precision but low recall is just the opposite, returning very few results, but most of its predicted 
   labels are correct when compared to the training labels. An ideal system with high precision and high recall will return 
   many results, with all results labeled correctly.

Precision (P) is defined as the number of true positives (T_p) over the number of true positives plus the number of false positives (F_p).

 P = {T_p} / {T_p+F_p}

Recall (R) is defined as the number of true positives (T_p) over the number of true positives plus the number of false negatives (F_n).
 
 R = {T_p} / {T_p + F_n}
 
These quantities are also related to the (F_1) score, which is defined as the harmonic mean of precision and recall.
 
 F1 = 2*{P * R} / {P+R}

  


In [28]:
## Code for visualizing the results of the precision-recall analysis
## http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
 
from itertools import cycle
# setup plot details, there are seven classes in the dataset, thus seven colors
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal', 'dimgray', 'darkviolet'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))


plt.show()  

NameError: name 'plt' is not defined

## Other ideas for the lab
https://hackernoon.com/transfer-learning-with-mxnet-gluon-8203005afafe